In [2]:
import pandas as pd
import geopandas as gpd

import sys
sys.path.append('../scripts/')
from read_utils import read_file, create_folder, temp_record_query, temp_record_sdf
from pyspark.sql import SparkSession

In [3]:
spark = (
    # Create a spark session (which will run spark jobs)
    SparkSession.builder.appName("Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config('spark.executor.memory','10g')
    .config('spark.driver.memory','12g')
    .config('spark.driver.maxResultsSize', '10GiB')
    # .config("spark.network.timeout", "3600s")
    # .master("local[6]")
    .getOrCreate()
    )

22/09/26 20:41:14 WARN Utils: Your hostname, Runyus-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.3.12 instead (on interface en0)
22/09/26 20:41:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/26 20:41:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [26]:
POA_df = pd.read_csv('../data/tables/au_postcodes.csv')
pos_df = POA_df.drop_duplicates(['postcode'])
POA_geo = gpd.GeoDataFrame(data = pos_df)

In [78]:
sa2_sf = gpd.read_file('../data/SA2_2021_AUST_SHP_GDA2020/SA2_2021_AUST_GDA2020.shp')
sa2_sf['geometry'] = sa2_sf['geometry'].to_crs("+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs")
sa2_gdf = gpd.GeoDataFrame(sa2_sf)
sa2_gdf = sa2_gdf.dropna()
sa2_gdf['centroid'] = sa2_gdf['geometry'].apply(lambda x: (x.centroid.y, x.centroid.x))
bound = sa2_gdf['geometry'].bounds
sa2_geo = sa2_gdf.join(bound)

In [92]:
sa2_geo_info = sa2_geo[['SA2_CODE21', 'geometry','centroid']].drop_duplicates('SA2_CODE21')

In [60]:
def match_sa2_and_postcode(sa2_df, pos_df):
    # pos_new_df = pos_df.reindex(columns=[*pos_df.columns.tolist(), 'SA2_CODE21'], fill_value=0)
    for i in pos_df.index:
        for j in sa2_df.index:
            if ((pos_df['latitude'][i] >= sa2_df['miny'][j])& (pos_df['latitude'][i] <= sa2_df['maxy'][j])
                &(pos_df['longitude'][i] >= sa2_df['minx'][j]) & (pos_df['longitude'][i] <= sa2_df['maxx'][j])):
                pos_df.loc[i, 'SA2_CODE21'] = sa2_df['SA2_CODE21'][j]
    return pos_df

In [61]:
mix_df = match_sa2_and_postcode(sa2_geo, POA_geo)

In [72]:
mix_df = mix_df.dropna()

In [73]:
mix_df.dtypes

postcode        int64
place_name     object
state_name     object
state_code     object
latitude      float64
longitude     float64
accuracy      float64
SA2_CODE21     object
dtype: object

In [74]:
mix_df.loc[:,'SA2_CODE21'] = mix_df.loc[:,'SA2_CODE21'].astype('int')

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [81]:
mix_df.dtypes

postcode        int64
place_name     object
state_name     object
state_code     object
latitude      float64
longitude     float64
accuracy      float64
SA2_CODE21      int64
dtype: object

In [93]:
sa2_geo_info.loc[:,'SA2_CODE21'] = sa2_geo_info.loc[:,'SA2_CODE21'].astype('int')

In [83]:
sa2_geo_info.dtypes

SA2_CODE21       int64
geometry      geometry
dtype: object

In [94]:
final_df = pd.merge(mix_df, sa2_geo_info, on='SA2_CODE21',how='inner')

In [99]:
final_df.head(5)

,postcode,place_name,state_name,state_code,latitude,longitude,accuracy,SA2_CODE21,geometry,centroid
0,200,Australian National University,Australian Capital Territory,ACT,-35.2777,149.1189,1.0,801051049,"POLYGON ((149.11002 -35.28400, 149.10987 -35.2...","(-35.281339155480744, 149.11505674306417)"
1,221,Barton,Australian Capital Territory,ACT,-35.3049,149.1412,4.0,801061129,"POLYGON ((149.12961 -35.30890, 149.12962 -35.3...","(-35.30925425962516, 149.13585270514673)"
2,2540,Wreck Bay,Australian Capital Territory,ACT,-35.1627,150.6907,4.0,901031003,"MULTIPOLYGON (((150.69567 -35.18295, 150.69556...","(-35.15235858081164, 150.6912049877714)"
3,2600,Deakin West,Australian Capital Territory,ACT,-35.3126,149.1278,3.0,801061063,"POLYGON ((149.11789 -35.31529, 149.11887 -35.3...","(-35.3171832215981, 149.12476387791898)"
4,2601,City,Australian Capital Territory,ACT,-35.2813,149.1293,4.0,801051053,"POLYGON ((149.12464 -35.27583, 149.12474 -35.2...","(-35.28126033741348, 149.12916957590815)"


In [100]:
final_df.to_csv('../data/curated/sa2_pos_geometry.csv')

In [102]:
geometry = read_file(spark, '../curated/sa2_pos_geometry.csv',type='csv')

|> Loading File...
|> Loading Finished!
22/09/26 21:58:04 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , postcode, place_name, state_name, state_code, latitude, longitude, accuracy, SA2_CODE21, geometry, centroid
 Schema: _c0, postcode, place_name, state_name, state_code, latitude, longitude, accuracy, SA2_CODE21, geometry, centroid
Expected: _c0 but found: 
CSV file: file:///Users/runyuyang/Downloads/2022S2/generic-buy-now-pay-later-project-group-24/data/curated/sa2_pos_geometry.csv
-RECORD 0--------------------------------------------------------------------------------------
 _c0        | 0                                                                                
 postcode   | 200                                                                              
 place_name | Australian National University                                                   
 state_name | Australian Capital Territory                                                     
 